# **Reviews prediction**

In [ ]:
!pip install gensim
!pip install scikeras
!pip install keras-tuner

In [ ]:
#imports

import urllib.request
import zipfile
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import re
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding, Dropout, Input
from sklearn.metrics import f1_score
from tensorflow.keras.initializers import he_normal
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from kerastuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam
from keras_tuner import Objective
from kerastuner.tuners import RandomSearch
import shutil
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer

Uploading the dataset

In [ ]:
# ZIP data URL from GitHub
zip_url = "https://github.com/sandyj02/projects/raw/main/DeepLearningProject/input.zip"

# Download the ZIP
zip_path, _ = urllib.request.urlretrieve(zip_url)

# Unzip the folder
target_folder = "temp"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

data_path = f"{target_folder}/input.pkl"

import pickle as pk

# Open the pickle data
with open(data_path, 'rb') as f:
  train_labels, test_labels, train_texts, test_texts, train_bow, test_bow = pk.load(f)

I'm going to start by taking a look at some of the data that we are dealing with.

2) I'm taking into account only text as a input since I deemed the bag of words field to be inefficient because of its sparsity.



In [ ]:
#taking a look at the length of the train and test sets
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(560, 560, 140, 140)

In [ ]:
#taking a look at how many instances belong to positive or negative class
count_pos, count_neg= 0, 0

for i in train_labels:
  if i=='positive':
    count_pos+=1
  else:
    count_neg+=1

print(f"class 1: {count_pos} \nclass 0: {count_neg}")

class 1: 398 
class 0: 162


In [ ]:
#taking at look at samples of data
for i in range (4):
  print(train_labels[i] + " | " + train_texts[i])

positive | Nifty little episode played mainly for laughs, but with clever dollop of suspense. Somehow a Martian has snuck aboard a broken-down bus on its way to nowhere, but which passenger is it, (talk about your illegal immigrants!). All-star supporting cast, from wild-eyed Jack Elam (hamming it up shamelessly), to sexy Jean Willes (if she's the Martian, then I say let's open the borders!), to cruel-faced John Hoyt (the most obvious suspect), along with familiar faces John Archer and Barney Phillips (and a nice turn from Bill Kendis as the bus driver). Makes for a very entertaining half-hour even if the action is confined to a single set.
positive | this is one of the finest movies i have ever seen....the stark scenery...the isolation...the ignorant bigoted people hiding behind their religion...a backdrop for some wordliness and sophistication...the acting is completely natural...but for me as a"foodie' the best is the actual choosing and preparation of the feast..i have spent time i

# Data Preprocessing

This was not mentioned in my exam but since i noticed some html tags i'm going to delete them since the model may not know how to preprocess them

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

def html_tags_removal(text_list):
  stripped_text=[]
  for i in text_list:
    soup = BeautifulSoup(i, "html.parser")
    stripping = soup.get_text(separator=" ")
    stripped_text.append(stripping)
  return stripped_text

train_stripped_text=html_tags_removal(train_texts)
test_stripped_text=html_tags_removal(test_texts)

#examples text of before and after tags removal
print(f"original: {train_texts[2]} \ncorrected: {train_stripped_text[2]}\n")
print(f"original: {test_texts[0]} \ncorrected: {test_stripped_text[0]}")

original: I've read the other reviews and found some to be comparison of movie v real life (eg what it takes to get into music school), Britney Bashing, etc, etc. so let's focus on the movie and the message.<br /><br />I have rated this movie 7 out of 10 for the age range 8 to 14 years, and for a family movie. For the average adult male.... 2 out of 10.<br /><br />I like pop/rock music, i'm 45. I know of Britney Spears but never realised she actually sang Stronger until i read the credits and these reviews. I didn't recognise her poster on the wall so I was not worried about any 'self promotion'.<br /><br />I watch movies to be entertained. i don't care about casting, lighting, producers, directors, etc. What is the movie and does it entertain me.<br /><br />I watched this movie for the message. The world's greatest epidemic is low self-esteem (which is a whole other story) so watched with the message in mind, as that is an area of interest. The movie is light, bright and breezy, great

2) Now I perform tokenization and punctuation removal of the text with simple regex

In [ ]:
def tokenize_remove_punctuation(text_list):
  tokenized_text = []
  for i in text_list:
      tokens = re.split('\W+', i)
      tokenized_text.append(tokens)
  return tokenized_text

train_tokenized_text = tokenize_remove_punctuation(train_stripped_text)
test_tokenized_text = tokenize_remove_punctuation(test_stripped_text)

#examples text after tokenization and punctuation removal
print(train_tokenized_text[2])
print(test_tokenized_text[0])

['I', 've', 'read', 'the', 'other', 'reviews', 'and', 'found', 'some', 'to', 'be', 'comparison', 'of', 'movie', 'v', 'real', 'life', 'eg', 'what', 'it', 'takes', 'to', 'get', 'into', 'music', 'school', 'Britney', 'Bashing', 'etc', 'etc', 'so', 'let', 's', 'focus', 'on', 'the', 'movie', 'and', 'the', 'message', 'I', 'have', 'rated', 'this', 'movie', '7', 'out', 'of', '10', 'for', 'the', 'age', 'range', '8', 'to', '14', 'years', 'and', 'for', 'a', 'family', 'movie', 'For', 'the', 'average', 'adult', 'male', '2', 'out', 'of', '10', 'I', 'like', 'pop', 'rock', 'music', 'i', 'm', '45', 'I', 'know', 'of', 'Britney', 'Spears', 'but', 'never', 'realised', 'she', 'actually', 'sang', 'Stronger', 'until', 'i', 'read', 'the', 'credits', 'and', 'these', 'reviews', 'I', 'didn', 't', 'recognise', 'her', 'poster', 'on', 'the', 'wall', 'so', 'I', 'was', 'not', 'worried', 'about', 'any', 'self', 'promotion', 'I', 'watch', 'movies', 'to', 'be', 'entertained', 'i', 'don', 't', 'care', 'about', 'casting', 

2) Now we perform case folding

In [ ]:
def case_folding(text):
  case_folding_list = []
  for sentence in text:
      folded_sentence = [word.lower() for word in sentence]
      case_folding_list.append(folded_sentence)
  return case_folding_list

train_lower=case_folding(train_tokenized_text)
test_lower=case_folding(test_tokenized_text)

#examples of case foleded text
print(train_lower[2])
print(test_lower[0])

['i', 've', 'read', 'the', 'other', 'reviews', 'and', 'found', 'some', 'to', 'be', 'comparison', 'of', 'movie', 'v', 'real', 'life', 'eg', 'what', 'it', 'takes', 'to', 'get', 'into', 'music', 'school', 'britney', 'bashing', 'etc', 'etc', 'so', 'let', 's', 'focus', 'on', 'the', 'movie', 'and', 'the', 'message', 'i', 'have', 'rated', 'this', 'movie', '7', 'out', 'of', '10', 'for', 'the', 'age', 'range', '8', 'to', '14', 'years', 'and', 'for', 'a', 'family', 'movie', 'for', 'the', 'average', 'adult', 'male', '2', 'out', 'of', '10', 'i', 'like', 'pop', 'rock', 'music', 'i', 'm', '45', 'i', 'know', 'of', 'britney', 'spears', 'but', 'never', 'realised', 'she', 'actually', 'sang', 'stronger', 'until', 'i', 'read', 'the', 'credits', 'and', 'these', 'reviews', 'i', 'didn', 't', 'recognise', 'her', 'poster', 'on', 'the', 'wall', 'so', 'i', 'was', 'not', 'worried', 'about', 'any', 'self', 'promotion', 'i', 'watch', 'movies', 'to', 'be', 'entertained', 'i', 'don', 't', 'care', 'about', 'casting', 

2) And finally as the last step of preprocessing let's perform word embeddings with word2vec. By doing so we obtain a numerical representation of the text to feed to the model

In [ ]:
model = gensim.models.Word2Vec(train_lower, vector_size=200, window=10, min_count=2)

def get_sentence_embedding(sentence_tokens, model):
  word_embeddings = [model.wv[word] for word in sentence_tokens if word in model.wv]
  if not word_embeddings:
      return np.zeros(model.vector_size)
  else:
      return np.mean(word_embeddings, axis=0)

train_sentence_embeddings = [get_sentence_embedding(sentence, model) for sentence in train_lower]
test_sentence_embeddings = [get_sentence_embedding(sentence, model) for sentence in test_lower]

X_train = np.array(train_sentence_embeddings)
X_test = np.array(test_sentence_embeddings)

X_train, X_test

(array([[ 0.06798946, -0.01916741, -0.03105113, ..., -0.4840906 ,
         -0.07015246, -0.24227408],
        [ 0.08075116, -0.02504006, -0.03707635, ..., -0.5736068 ,
         -0.08399564, -0.29066166],
        [ 0.07056703, -0.01964231, -0.04766725, ..., -0.55007607,
         -0.07300784, -0.2992239 ],
        ...,
        [ 0.07738679, -0.02201782, -0.04605547, ..., -0.5793911 ,
         -0.07993756, -0.30869502],
        [ 0.07370503, -0.02138978, -0.04088891, ..., -0.54972833,
         -0.07675394, -0.28723934],
        [ 0.07827501, -0.02214917, -0.04601169, ..., -0.58095205,
         -0.0788158 , -0.30790952]], dtype=float32),
 array([[ 0.08025537, -0.02635876, -0.0316317 , ..., -0.54715   ,
         -0.08206019, -0.2724247 ],
        [ 0.08257303, -0.02382403, -0.03927889, ..., -0.59153897,
         -0.08506055, -0.3003761 ],
        [ 0.06673774, -0.01721528, -0.0496358 , ..., -0.53053546,
         -0.06690422, -0.29327458],
        ...,
        [ 0.07260589, -0.01967866, -0.0

3) Now let's convert the labels lists into binary values (positive=1, negative=0)

In [ ]:
def convert_labels_to_binary(labels):
  binary_labels = [1 if label == 'positive' else 0 for label in labels]
  return binary_labels

binary_train=convert_labels_to_binary(train_labels)
binary_test=convert_labels_to_binary(test_labels)

Y_train=np.array(binary_train)
Y_test=np.array(binary_test)

print (train_labels[:5])
print (Y_train[:5])
print (test_labels[:5])
print (Y_test[:5])

['positive', 'positive', 'positive', 'negative', 'negative']
[1 1 1 0 0]
['positive', 'negative', 'positive', 'positive', 'positive']
[1 0 1 1 1]


# Model Training


1) The model that I'm going to use to make prediction on the reviews is bidirectional reccurent neural network

In [ ]:
# Reshaping
num_train_samples, embedding_dim = X_train.shape
X_train_reshaped = X_train.reshape((num_train_samples, 1,  embedding_dim))
num_test_samples, embedding_dim = X_test.shape
X_test_reshaped = X_test.reshape((num_test_samples, 1,  embedding_dim))

5)
 - The model will be composed of bidirectional rnn layers and dense layers
 - I will use ReLu as an activation function for the hidden layers
 - I will use drouput for regularization and i will be using HE initializer since it works well with  ReLU

In [ ]:
random_seed = 10
tf.random.set_seed(random_seed)

def f1_metric(y_true, y_pred):
  y_pred_binary = tf.cast(y_pred > 0.5, tf.int32)
  return tf.py_function(f1_score, (y_true, y_pred_binary), tf.float64)

model = Sequential([
  Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), input_shape=( X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
  Dropout(0.5),
  Bidirectional(LSTM(256, kernel_initializer='he_normal', return_sequences=True)),
  Dropout(0.5),
  Bidirectional(LSTM(128, kernel_initializer='he_normal')),
  Dropout(0.5),
  Dense(64, activation='relu', kernel_initializer='he_normal'), #I'm using re_lu as an activation function for hidden layers to introduce non linearity
  Dropout(0.5),
  Dense(1, activation='sigmoid')  #3)The output layer has sigmoid as an activation function
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_metric])
model.fit(X_train_reshaped, Y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, Y_test)) #5)epochs and batch_size manually tuned

Epoch 1/10
18/18 [==============================] - 19s 384ms/step - loss: 0.6363 - f1_metric: 0.8191 - val_loss: 0.5878 - val_f1_metric: 0.8436
Epoch 2/10
18/18 [==============================] - 1s 42ms/step - loss: 0.6140 - f1_metric: 0.8274 - val_loss: 0.5891 - val_f1_metric: 0.8436
Epoch 3/10
18/18 [==============================] - 1s 42ms/step - loss: 0.6108 - f1_metric: 0.8282 - val_loss: 0.5891 - val_f1_metric: 0.8436
Epoch 4/10
18/18 [==============================] - 1s 41ms/step - loss: 0.6156 - f1_metric: 0.8320 - val_loss: 0.5846 - val_f1_metric: 0.8436
Epoch 5/10
18/18 [==============================] - 1s 42ms/step - loss: 0.6119 - f1_metric: 0.8255 - val_loss: 0.5877 - val_f1_metric: 0.8436
Epoch 6/10
18/18 [==============================] - 1s 49ms/step - loss: 0.6066 - f1_metric: 0.8319 - val_loss: 0.5846 - val_f1_metric: 0.8436
Epoch 7/10
18/18 [==============================] - 1s 41ms/step - loss: 0.6055 - f1_metric: 0.8303 - val_loss: 0.5904 - val_f1_metric: 0.84

There seems to be very little changes for each epoch, let's see if we can get a better result with hyperparameter tuning.

# Hyperprameter tuning


5) I will be performing hyperparameter tuning on learning rate, batch size, dropout rate and number of layers using keras tuner

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
  model.add(Dropout(hp.Float('dropout_1', 0, 0.7, step=0.1))) #Tuning dropout rate

  # Tuning number of layers
  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal')))
    model.add(Dropout(hp.Float('dropout_' + str(i+1), 0, 0.7, step=0.1))) #Tuning dropout rate

  model.add(Bidirectional(LSTM(128, kernel_initializer='he_normal')))
  model.add(Dropout(hp.Float('dropout_last', 0, 0.7, step=0.1))) #Tuning dropout rate
  model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
  model.add(Dropout(hp.Float('dropout_dense', 0, 0.7, step=0.1))) #Tuning dropout rate
  model.add(Dense(1, activation='sigmoid'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=['accuracy']) #Tuning learning rate

  return model

In [ ]:

def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(Dropout(hp.Float('dropout_1', 0, 0.7, step=0.1))) #Tuning dropout rate

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal')))
        model.add(Dropout(hp.Float('dropout_' + str(i+1), 0, 0.7, step=0.1))) #Tuning dropout rate

    model.add(Bidirectional(LSTM(128, kernel_initializer='he_normal')))
    model.add(Dropout(hp.Float('dropout_last', 0, 0.7, step=0.1))) #Tuning dropout rate
    model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(hp.Float('dropout_dense', 0, 0.7, step=0.1))) #Tuning dropout rate
    model.add(Dense(1, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=['accuracy']) #Tuning learning rate

    return model

shutil.rmtree('my_dir/my_project', ignore_errors=True)

tuner = kt.tuners.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='my_project'
)


def train_model(model, x_train, y_train, x_val, y_val, epochs, batch_size):
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    return history


tuner.search(x=X_train_reshaped, y=Y_train,
             epochs=10,
             batch_size=64,
             validation_data=(X_test_reshaped, Y_test),
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])


best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

NameError: name 'kt' is not defined

Hyperparameter tuning didn't seem to help the performance

6) To take a look at the overall performance of the model I'm going to evaluate it based on accuracy, precision, recall and f1_score

In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)

Y_pred = best_model.predict(X_test_reshaped)
Y_pred_classes = (Y_pred > 0.5).astype("int32")

accuracy = accuracy_score(Y_test, Y_pred_classes)
precision = precision_score(Y_test, Y_pred_classes)
recall = recall_score(Y_test, Y_pred_classes)
f1 = f1_score(Y_test, Y_pred_classes)

print(f"Accuracy: {accuracy}\n Precision:  {precision}\n Recall: {recall}\n F1 Score:{f1}")

6) Know we are going to check the performance on stratified k fold cross validation

In [ ]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
f1_scores = []

for train_idx, test_idx in kf.split(X_train_reshaped, Y_train):
  X_train_fold, X_test_fold = X_train_reshaped[train_idx], X_train_reshaped[test_idx]
  Y_train_fold, Y_test_fold = Y_train[train_idx], Y_train[test_idx]

  best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
  best_model = tuner.hypermodel.build(best_hp)

  best_model.fit(X_train_fold, Y_train_fold, epochs=10, batch_size=32)

  Y_pred_fold = best_model.predict(X_test_fold)
  Y_pred_classes_fold = (Y_pred_fold > 0.5).astype("int32")

  f1_fold = f1_score(Y_test_fold, Y_pred_classes_fold)

  f1_scores.append(f1_fold)

mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)


print(f"Mean F1 Score: {mean_f1}")
print(f"Standard Deviation of F1 Scores: {std_f1}")

Validation scores seem to be decently high but while running the model it seems that with each epoch the score did not have a lot of variation something that were not mentioned in the exam that could be implemented are: changing model, adding preprocessing steps such as stop word removal, stemming/lemmatizing, name entity recognition to help the classification process